# Neuron Simulator

**University of Pavia**

*Brain Modelling*

**Michele Ventimiglia**

---

## Setup

Install required libraries:

In [ ]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: '-q'


Libraries import:

In [14]:
import sys
from typing import Tuple, Dict, List

import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact, FloatSlider, Button

Versions info:

In [15]:
print(f"Python Version:\t{sys.version.split()[0]}")
print(f"NumPy Version:\t{np.__version__}")
print(f"SciPy Version:\t{scipy.__version__}")

Python Version:	3.12.10
NumPy Version:	2.2.6
SciPy Version:	1.15.3


---

## Model

Implement the Morris-Lecar model to calculate the time evolution of membrane potential (V) and gating variable (w) in a neuron based on differential equations.

The Morris-Lecar model is a simplified model of neuronal activity that describes how the membrane potential evolves over time in response to external currents and the neuron's own ion channels.

In [16]:
def morris_lecar(
        V_w: Tuple[float, float],
        t: float,
        params: Dict[str, float]
    ) -> Tuple[float, float]:
    """
    Morris-Lecar model equations to compute the time evolution of membrane potential (V) and gating variable (w).\n
    ---
    ### Args
    - `V_w` (`Tuple[float, float]`): a tuple containing the current membrane potential (V) and the gating variable (w).
    - `t` (`float`): the current time point (not used in the equations but required by odeint).
    - `params` (`Dict[str, float`]): a dictionary containing the model parameters such as `I_ext`, `g_Ca`, `g_K`, `g_L`, `C`, `V_Ca`, `V_K`, `V_L`, `V1`, `V2`, `V3`, `V4`, and `phi`.\n
    ---
    ### Returns
    - `Tuple[float, float]`: a tuple containing the derivatives of the membrane potential (dV/dt) and the gating variable (dw/dt).
    """
    
    V, w = V_w # Unpack the membrane potential and gating variable
    I_ext = params['I_ext']
    
    # Calculate the steady-state activation and time constant for the gating variable
    m_inf = 0.5 * (1.0 + np.tanh((V - params['V1']) / params['V2']))
    w_inf = 0.5 * (1.0 + np.tanh((V - params['V3']) / params['V4']))
    tau_w = 1 / (params['phi'] * np.cosh((V - params['V3']) / (2 * params['V4'])))
    
    # Calculate the derivatives of V and w using the Morris-Lecar equations
    dV_dt = (I_ext - params['g_Ca'] * m_inf * (V - params['V_Ca']) - params['g_K'] * w * (V - params['V_K']) - params['g_L'] * (V - params['V_L'])) / params['C']
    dw_dt = params['phi'] * (w_inf - w) / tau_w
    
    return [dV_dt, dw_dt]

### Parameters

In [17]:
def default_parameters() -> dict:
    """
    Default parameters and initial conditions for the Morris-Lecar model.\n
    ---
    ### Returns
    - `dict`: a dictionary containing the default parameters for the Morris-Lecar model.
    """
    return {
        "C": 20,            # Membrane capacitance, uF/cm^2
        "V_Ca": 120,        # Reversal potential for calcium, mV
        "V_K": -84,         # Reversal potential for potassium, mV
        "V_L": -60,         # Reversal potential for leak channels, mV
        "g_Ca": 4.4,        # Maximum conductance for calcium, mS/cm^2
        "g_K": 8,           # Maximum conductance for potassium, mS/cm^2
        "g_L": 2,           # Maximum conductance for leak, mS/cm^2
        "phi": 0.04,        # Temperature coefficient
        "V1": -1.2,         # Parameter for activation/inactivation
        "V2": 18,           # Parameter for activation/inactivation
        "V3": 2,            # Parameter for activation/inactivation
        "V4": 30,           # Parameter for activation/inactivation
        "I_ext": 0.1,       # External current, uA/cm^2
        "V0": -70*(10**-3), # Initial membrane potential, V
        "w0": 0.0           # Initial fraction of open potassium channels
    }

---

## Processing

Define the simulation function to compute the time evolution of the membrane potential (V) and gating variable (w) in a neuron based on the Morris-Lecar model.

In [18]:
def solve_model(I_ext: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Solve the Morris-Lecar model ODEs for a given external current (I_ext).\n
    ---
    ### Args
    - `I_ext` (`float`): the external current applied to the neuron, in uA/cm^2.\n
    ---
    ### Returns
    - `Tuple[np.ndarray, np.ndarray, np.ndarray]`: a tuple containing:
        - `t` (`np.ndarray`): the time vector over which the ODEs are solved.
        - `V` (`np.ndarray`): the membrane potential over time.
        - `w` (`np.ndarray`): the gating variable over time.
    """
    
    # Define the time vector over which to solve the ODEs
    t = np.linspace(0, 5000, 10000)
    
    # Retrieve default parameters and update them with the given values
    params = default_parameters()
    params.update({'I_ext': I_ext})
    
    # Define the initial conditions for the integration
    initial_conditions = [params['V0'], params['w0']]
    
    # Solve the ODEs using scipy's odeint function
    sol = odeint(morris_lecar, initial_conditions, t, args=(params,))
    
    # Return the time vector, membrane potential, and recovery variable
    return t, sol[:, 0], sol[:, 1]

Define the analyze data function to analyze the time evolution of the membrane potential (V) and gating variable (w) in a neuron based on the Morris-Lecar model.

In [19]:
def analyze_data(
        V: np.ndarray,
        w: np.ndarray,
        t: np.ndarray
    ) -> List[Dict[str, float]]:
    """
    Analyze the data to find critical points where the membrane potential (V) crosses the gating variable (w).\n
    ---
    ### Args
    - `V` (`np.ndarray`): the membrane potential over time.
    - `w` (`np.ndarray`): the gating variable over time.
    - `t` (`np.ndarray`): the time vector corresponding to the V and w arrays.\n
    ---
    ### Returns
    - `List[Dict[str, float]]`: a list of dictionaries containing the critical points where the membrane potential crosses the gating variable, with each dictionary containing:
        - `V` (`float`): the membrane potential at the crossing point.
        - `w` (`float`): the gating variable at the crossing point.
        - `t` (`float`): the time at which the crossing occurs.
    """
    # Calculate differences between V and w to find where they cross
    differences = V - w
    
    # Identifying the sign changes in differences, which indicate crossing points
    sign_changes = np.diff(np.sign(differences)) != 0

    # Getting the indices right before the crossing points
    cross_indices = np.where(sign_changes)[0]
    
    # Collecting critical points (V, w, t) at crossing points
    critical_points = [{'V': V[i], 'w': w[i], 't': t[i]} for i in cross_indices]

    return critical_points

---

## Evaluation

In [20]:
global current_plot_state

Define the plotting function to visualize the time evolution of the membrane potential (V) and gating variable (w) in a neuron based on the Morris-Lecar model.

In [25]:
def save_plot(filename: str = 'plot') -> None:
    
    global current_plot_state
    
    if current_plot_state is None:
        print("No plot to save.")
        return
    
    I_ext, V, w, t = current_plot_state
    
    filename = f"{filename}_I_ext_{I_ext}.png"
    
    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    plt.plot(t, V)
    plt.title('Membrane Potential Over Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.grid()

    plt.subplot(1, 3, 2)
    plt.plot(V, w)
    plt.title('Phase Plane')
    plt.xlabel('Membrane Potential (mV)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    plt.subplot(1, 3, 3)
    plt.plot(t, w)
    plt.title('Recovery Variable Over Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    plt.savefig(f"images/{filename}")
    plt.close()
    
    print(f"Plot saved as {filename}")

In [26]:
def interactive_plot(I_ext: float) -> None:
    
    global current_plot_state
    
    t, V, w = solve_model(I_ext)
    critical_points = analyze_data(V, w, t)
    
    current_plot_state = I_ext, V, w, t
    
    # Plotting
    plt.figure(figsize=(18, 6))

    # Plot V(t)
    plt.subplot(1, 3, 1)
    plt.plot(t, V)
    plt.title('Membrane Potential Over Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.grid()

    # Plot phase plane trajectory
    plt.subplot(1, 3, 2)
    plt.plot(V, w)
    plt.title('Phase Plane')
    plt.xlabel('Membrane Potential (mV)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()

    # Plot w(t)
    plt.subplot(1, 3, 3)
    plt.plot(t, w)
    plt.title('Recovery Variable Over Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Recovery Variable (w)')
    plt.grid()
    
    plt.show()

    # Print analysis results
    for i, point in enumerate(critical_points):
        if i != 0: # skip initial graphs value
            print(f"Critical point {i}: {point}")


Instantiate the simulation function with the parameters and initial conditions, then run the simulation:

In [27]:
params = default_parameters()

Interactive widgets to change parameters and visualize the results:

In [28]:
interact(
    interactive_plot,
    I_ext = FloatSlider(
        value = params['I_ext'],
        min = 0,
        max = 100,
        step = 10))

save_button = Button(description="Save Plot")
save_button.on_click(lambda b: save_plot())
display(save_button)

interactive(children=(FloatSlider(value=0.1, description='I_ext', step=10.0), Output()), _dom_classes=('widget…

Button(description='Save Plot', style=ButtonStyle())